<a href="http://landlab.github.io"><img style="float: left" src="../../media/landlab_header.png"></a>

# Large scale landscape evolution model with Priority flood flow router and Space_v2
<hr>

The priority flood flow director is designed to calculate flow properties over large scale grids. In the following notebook we illustrate how the priority flood flow accumulator can be used to simulate landscape evolution using the SPAVE_V2 Landlab component

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import time

from landlab import imshow_grid, RasterModelGrid
from landlab.components import (
    FlowAccumulator,
    DepressionFinderAndRouter,
    Space,
    SpaceLargeScaleEroder,
    PriorityFloodFlowRouter,
)

Create raster grid

In [ ]:
# nr = 20
# nc = 20
nr = 75
nc = 75
xy_spacing = 10.0
mg = RasterModelGrid((nr, nc), xy_spacing=xy_spacing)
z = mg.add_zeros("topographic__elevation", at="node")
mg.at_node["topographic__elevation"][mg.core_nodes] += np.random.rand(
    mg.number_of_core_nodes
)

s = mg.add_zeros("soil__depth", at="node", dtype=float)
mg.at_node["soil__depth"][mg.core_nodes] += 0.5
mg.at_node["topographic__elevation"] += mg.at_node["soil__depth"]

fr = FlowAccumulator(mg, flow_director='D8')
df = DepressionFinderAndRouter(mg)

ha = Space(mg, K_sed=0.00005, K_br=0.00005, phi=0.3, H_star=1)

br = mg.at_node["bedrock__elevation"]
z = mg.at_node["topographic__elevation"]

space_dt = 500

In [ ]:
z_ori = np.array(z)
t1 = time.time()
for i in tqdm(range(50)):
    # Uplift
    br[mg.core_nodes] += 0.001 * space_dt
    z[mg.core_nodes] = br[mg.core_nodes] + s[mg.core_nodes]
    fr.run_one_step()
    df.map_depressions()
    ha.run_one_step(dt=space_dt)

t_span1 = time.time() - t1
print('Total run time is %.f s' %t_span1)

In [ ]:
plt.figure(figsize=(10,10))
imshow_grid(mg, "topographic__elevation", cmap="terrain")
plt.title("Final topographic__elevation")

In [ ]:
mg2 = RasterModelGrid((nr, nc), xy_spacing=xy_spacing)
z2 = mg2.add_zeros("topographic__elevation", at="node")
mg2.at_node["topographic__elevation"][mg2.core_nodes] += np.random.rand(
    mg2.number_of_core_nodes
)

s2 = mg2.add_zeros("soil__depth", at="node", dtype=float)
mg2.at_node["soil__depth"][mg2.core_nodes] += 0.5
mg2.at_node["topographic__elevation"] += mg2.at_node["soil__depth"]

fr2 = PriorityFloodFlowRouter(mg2, flow_metric="D8", update_flow_depressions=True)

ha2 = SpaceLargeScaleEroder(mg2, K_sed=0.00005, K_br=0.00005, phi=0.3, H_star=1)

br2 = mg2.at_node["bedrock__elevation"]
z2 = mg2.at_node["topographic__elevation"]

In [ ]:
z_ori = np.array(z2)
t2 = time.time()
for i in tqdm(range(50)):
    # Uplift
    br2[mg2.core_nodes] += 0.001 * space_dt
    z2[mg2.core_nodes] = br2[mg2.core_nodes] + s2[mg2.core_nodes]
    fr2.run_one_step()
    ha2.run_one_step(dt=space_dt)

t_span2 = time.time() - t2

print('Total run time is %.f s' %t_span2)

In [ ]:
plt.figure(figsize=(10,10))
imshow_grid(mg2, "topographic__elevation", cmap="terrain")
plt.title("Final topographic__elevation")

In [ ]:
plt.figure()
plt.bar(['Default flow accumulator','Priority Flood flow accumulator'],[t_span1,t_span2])
plt.ylabel('Seconds')

## Back to HyLands tutorial page
[Click here to go back to the tutorial overview page](../index.ipynb)